<a href="https://colab.research.google.com/github/Andrea-Gaudio/DnD_Monster_Creator/blob/master/TestMatrixMult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sparse Matrices for PageRank
As per problem description, we attempt an implementation of a sparse matrix for the calculation of PageRank.

We have selected the CSR compression method after the anlysis of the problem, as it involves several multiplications and no further modifications of the matrix structure down the line. We have also implemented the multiplication function between matrix and vector, as it will be necessary in the rest of the exercise

In order to be able to work with actual matrices, we have decided to utilize the numpy library, the rest of the functions and data structures will be implemented

In [ ]:
#imports of the numpy library

import numpy as np

# We will import also the standard libraries in order to provide a benchmark for our implementation

In [ ]:

# CSR matrix implementation
# The matrix is composed by 3 arrays, that contain the values, the column indices and the sum of the elements in the previous rows
# for convenience, it also contains the size of the matrix. We only implement the class for square matrices, so n == m

class CSRmat:
    AA : np.ndarray
    JA : np.ndarray
    IA : np.ndarray
    n  : int

    def __init__ (self, n:int, nz:int):
        self.AA = np.empty(nz, dtype = np.float64)
        self.JA = np.empty(nz, dtype = np.int32)
        self.IA = np.empty(n+1, dtype = np.int32)
        self.n = n

In [ ]:
# We implement the multiplication between a matrix and a vector by using the CSRmat class that was just created

def matMult(M: CSRmat, v:np.ndarray) -> np.ndarray:
    n = M.n
    IA = M.IA
    AA = M.AA
    JA = M.JA

    w = np.zeros(n, dtype = np.float64)
    for i in range(n):
        row_start = IA[i]
        row_end   = IA[i+1]

        if row_start < row_end:
            w[i] = np.dot(AA[row_start:row_end], v[JA[row_start:row_end]])

    return w

Here we implement the Graph class, to store the graph of the different pages for the pageRank algorithm

The graph is stored as a CSRmat object acting as an adjecency matrix.
To keep everythong concentrated, we also inserted inside the object a dictionary that contains the pairs Vertex-Label (in this case, URL address of the page), and the number of vertices V

We also implemented the method that loads the data in the graph, given the Edge List of the graph

In [ ]:
class SparseGraph():
    labels  : dict
    adjMat  : CSRmat
    V       : int

    def __init__(self, labels:dict, V:int, nz:int):
        self.labels = labels
        self.adjMat = CSRmat(V, nz)
        self.V = V


    def loadEdgeList(self, edgeList: list):
        edgeList.sort(key=lambda edge: int(edge.split()[0]))
        current_row = 0
        nnz_cnt = 0

        self.adjMat.IA[0] = 0

        for i, edge in enumerate(edgeList):
            a , b = map(int, edge.split())


            self.adjMat.AA[i] = 1.0
            self.adjMat.JA[i] = b

            if (a > current_row):
              for r in range(current_row + 1, a + 1):
                self.adjMat.IA[r] = i

              current_row = a

            nnz_cnt += 1

        for r in range(current_row + 1, self.adjMat.n + 1):
             self.adjMat.IA[r] = nnz_cnt

Confronto con le librerie standard

In [ ]:

N = 5 # Dimension (5x5 matrix)
NZ = 5 # Number of non-zero entries

# Edge list sorted by row index (source node)
edge_list = [
    "0 2",
    "1 1",
    "2 0",
    "2 4", # Row 2 has two entries
    "3 3"
]

# Instantiate the SparseGraph
graph = SparseGraph(labels={}, V=N, nz=NZ)

# Load the edges, which sets the structure (IA, JA)
graph.loadEdgeList(edge_list)

# Manually set the specific AA values corresponding to the matrix A
graph.adjMat.AA[0] = 2.0
graph.adjMat.AA[1] = 4.0
graph.adjMat.AA[2] = 1.0
graph.adjMat.AA[3] = 5.0
graph.adjMat.AA[4] = 3.0

# Define the vector v
v = np.array([1.0, 2.0, 3.0, 4.0, 5.0])

# Perform the matrix-vector multiplication
w = matMult(graph.adjMat, v)

# Verification
expected_w = np.array([6.0, 8.0, 26.0, 12.0, 0.0])

print("\n--- CSR Matrix Data (Internal State) ---")
print(f"AA (Data)  : {graph.adjMat.AA}")
print(f"JA (Indices): {graph.adjMat.JA}")
print(f"IA (Pointers): {graph.adjMat.IA}")
print("----------------------------------------")
print(f"Input Vector v: {v}")
print(f"Result Vector w: {w}")
print(f"Expected Vector: {expected_w}")

is_correct = np.allclose(w, expected_w)
print(f"\nVerification successful: **{is_correct}**")


--- CSR Matrix Data (Internal State) ---
AA (Data)  : [2. 4. 1. 5. 3.]
JA (Indices): [2 1 0 4 3]
IA (Pointers): [0 1 2 4 5 5]
----------------------------------------
Input Vector v: [1. 2. 3. 4. 5.]
Result Vector w: [ 6.  8. 26. 12.  0.]
Expected Vector: [ 6.  8. 26. 12.  0.]

Verification successful: **True**


In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

# Data for the A matrix: (Row Index, Column Index, Value)
rows = np.array([0, 1, 2, 2, 3])
cols = np.array([2, 1, 0, 4, 3])
data = np.array([2.0, 4.0, 1.0, 5.0, 3.0])
N = 5

# 1. Create the CSR Matrix directly from the data
A_scipy = csr_matrix((data, (rows, cols)), shape=(N, N))

# 2. Define the vector v
v = np.array([1.0, 2.0, 3.0, 4.0, 5.0])

# 3. Perform Matrix Multiplication using the @ operator
w_scipy = A_scipy @ v

print("SciPy Result (w):", w_scipy)
# Output: SciPy Result (w): [ 6. 8. 26. 12. 0.]

In [ ]:
def importFromFile()